In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-23 21:03:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-05-23 21:03:10 (6.62 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.us-east-2.amazonaws.com/turtle-amazon-auto/amazon_reviews_us_Automotive_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [ ]:
from pyspark.sql.functions import count
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28258386|             3|
|   18634862|             3|
|    2072312|             3|
|   33132797|            15|
|   14539137|             1|
|   14552054|             5|
|    9347763|             1|
|   43626894|             2|
|   27314089|            47|
|   36829268|             1|
|   24540309|             2|
|    8501205|             1|
|    4361670|             2|
|   30416085|             2|
|    8306731|             1|
|   11491871|             1|
|   46406103|             1|
|   18417791|             1|
|   51299913|             1|
|   20587872|             4|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
#products_df = df.select([]).drop_duplicates()

products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|9539751705|Tennessee Heavy D...|
|B00006JPBB|Ai HWH806 Factory...|
|B00009WNYB|Megaman X Ultimat...|
|B0000AQM5V|  Coghlan's Rainsuit|
|B0000AX5XB|Raider BCS-8B Del...|
|B0000AXQO9|Sierra Internatio...|
|B0000AXVCB|Meguiars M4364 Bo...|
|B0000AY9W6|Camco 44412 Wheel...|
|B0000AZ8PG|NGK (1052) B6HS-1...|
|B0000UUU82|Lund 18063 Interc...|
|B0000UUX9S|K&N 33-2056 High ...|
|B000182DO2|Flowmaster 430402...|
|B000182ETG|Auto Ventshade 77...|
|B0001EW4D6|Auto Ventshade 94...|
|B0001EXKQ6|Auto Ventshade 92...|
|B0001WW1NQ|Custom Accessorie...|
|B0001YIUZC|Sportsman Series ...|
|B00027GZV4|Sierra 18-3575 Bi...|
|B00029JAOQ|Holley 125-25 Sin...|
|B00029JW3A|Saddleman Premier...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df = df.select(["review_id","customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26NFAHINEWAEN|   40585445|B00O14U6JG|     324591176| 2015-08-31|
|R11VYFA4DS540J|   35153667|B002XM5GHA|     240909913| 2015-08-31|
| RIGUYCMXKEZHP|     399812|B00LDNKIKU|      89901919| 2015-08-31|
| RWLJNTB8UUUKV|   14847460|B00DX8CGLW|     922008390| 2015-08-31|
|R1BVB2VR7ID4I2|   47237951|B00HVON60Y|     489280038| 2015-08-31|
|R2PHZXYFAJ5V44|   40654272|B00EA0ZB7I|     923404740| 2015-08-31|
| R42ASMHJW3RU5|   12629531|B001EA0LJK|     337648628| 2015-08-31|
|R301L6RTZ5G2K8|     728336|B00LNLL9DM|     899082152| 2015-08-31|
| RLDLU22MNPA6Z|     661944|B007KCZD0I|     181309549| 2015-08-31|
|R3LTI24XQN1VR4|   14135539|B00IWGLF1M|     920530953| 2015-08-31|
|R33YTLJG32YC55|   23754397|B00AN5UEFS|     761037438| 2015-08-31|
|R2PWCB34ILEK0E|   47343549|B006GFLI0O|     460159260| 2015-08

In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])

vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cgzuevuocpgl.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Nabokov20!", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## Deliverable 2

In [ ]:
#filter the dataset for the total_votes = or > 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count = df.filter("total_votes >= 20")
vote_count.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36886257|R1T4FVSABO0IDP|B00V03D9KY|     966609438|Lukas Lk-7950 WD ...|      Automotive|          5|            8|         31|   N|                Y|      Great Value!!!|LK-7950 is an out...| 2015-08-31|
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|    

In [ ]:
#create a new DataFrame/table to retrieve all the rows where the # of helpful_votes divided by total_votes is = or > 50%.
over_50 = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
over_50.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [ ]:
#create a new DataFrame/table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
over_50_y = over_50.filter(over_50["vine"] == "Y")
over_50_y.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35560012| REKN4PLM5174P|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|          219|        225|   Y|                N|Head 2 Head, It's...|Having come into ...| 2015-07-08|
|         US|   52693410| RQOG8IIJX7CKB|B00P9OF04I|      44916596|Meguiar's Heavy D...|      Automotive|          4|    

In [ ]:
#create a new DataFrame/table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
over_50_n = over_50.filter(over_50["vine"] == "N")
over_50_n.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21035025|R397VXR1GAK6C9|B00IS941D2|     373170065|EAG 07-17 Jeep Wr...|      Automotive|          5|           21|         21|   N|                Y|Awesome bumper. I...|Awesome bumper. I...| 2015-08-31|
|         US|   11359139|R2H6KB9RHS17GA|B00VNBDWPK|     790984844|OPT7 Fluxbeam LED...|      Automotive|          3|    

In [ ]:
five_star = over_50.filter(over_50["star_rating"]== 5)
five_star.cache()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [ ]:
#Total Reviews
over_50.count()


24824

In [ ]:
#Total Paid Reviews
over_50_y.count()

82

In [ ]:
#Total UnPaid Reviews

over_50_n.count()

24742

In [ ]:
#Total 5 Star Reviews
five_star.count()

12840

In [ ]:
#Total Paid 5 Star Reviews
five_star.filter(five_star["vine"] == "Y").count()

33

In [ ]:
#Total UnPaid 5 Star Reviews
five_star.filter(five_star["vine"] == "N").count()

12807

In [ ]:
#Percentage of five star reviews
five_star.count() / over_50.count()

0.5172413793103449

In [ ]:
#Percentage of paid reviews
five_star.filter(five_star["vine"] == "Y").count()/over_50_y.count()

0.4024390243902439

In [ ]:
#percentage of unpaid reviews
five_star.filter(five_star["vine"] == "N").count()/over_50_n.count()

0.5176218575701237